# Carbon Use Efficiency of Marine Heterotrophs

In [1]:
import cobra
import os
import pandas as pd

In [4]:
# Local install of gem2cue
# Should only need to be run once
# !python -m pip install -e /home/helen/Documents/PhD/Segre\ Lab/GEM2CUE

Obtaining file:///home/helen/Documents/PhD/Segre%20Lab/GEM2CUE
  Running setup.py develop for GEM2CUE


In [2]:
import gem2cue.utils

## Set-up

In [3]:
# Read in the meta-data file as a data frame
meta_df = pd.read_csv('IAMM_models/strain_metadata.tsv', sep='\t')
meta_df

,strain,model_file,gc_content,genome_length,memote_score,purely_met_rxns
0,Bork,Bork-LB.xml,0.547278,3120143,0.87,0.6515
1,Dies,Dies-LB.xml,0.616316,4928223,0.70,0.5889
2,Hot1A3,Hot1A3-LB.xml,0.446714,4801807,0.71,0.5993
3,BS11,BS11-LB.xml,0.446078,4480507,0.71,0.6106
4,DE,DE-LB.xml,0.448687,4480937,0.70,0.6163
5,Cstat,Cstat-LB.xml,0.548907,2862233,0.71,0.5887
6,DFL12,DFL12-LB.xml,0.655246,4417868,0.71,0.6266
7,Kuro,Kuro-LB.xml,0.470156,3850372,0.87,0.6224
8,HP15,HP15-LB.xml,0.568599,4651725,0.87,0.6182
9,MMB1,MMB1-LB.xml,0.441275,4684316,0.70,0.5675


In [4]:
# For each row, make a Strain object with the model and metadata from the table
strain_list = []
for index, row in meta_df.iterrows():
    strain_list.append(gem2cue.utils.Strain(row['strain'],
                                            cobra.io.read_sbml_model(os.path.join('IAMM_models', 'models', row['model_file'])),
                                            {'GC Content': row['gc_content'],
                                            'Genome Length': row['genome_length'],
                                            'Memote Score': row['memote_score'],
                                            'Purely Metabolic Rxns': row['purely_met_rxns']}))

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


## Analysis

### Experiment 0: Unconstrained

In [11]:
# Make an experiment for each Strain
unconstrained_experiments = [gem2cue.utils.Experiment(strain) for strain in strain_list]

In [12]:
# Calculate CUE for all of the Experiments
for exp in unconstrained_experiments:
    exp.CUE(ex_nomenclature = {'C_e'})

In [13]:
# Pull out all of the CUE results
{exp.strain.name: exp.cue for exp in unconstrained_experiments}

{'Bork': 0.902403441064068,
 'Dies': 0.9686221560132082,
 'Hot1A3': 0.920737201394963,
 'BS11': 0.936910828774397,
 'DE': 0.9022178607994155,
 'Cstat': 0.9492155844123638,
 'DFL12': 0.9658093219636261,
 'Kuro': 0.9438306696383395,
 'HP15': 0.9490527145104481,
 'MMB1': 0.950380039479828,
 'Abie': 0.9467016899808219,
 'DSS-3': 0.9418195790866428,
 'atlcs': 0.8648487501483723,
 'eryth': 0.9194099089851129,
 'Pga11': 0.8681636803969982,
 'Phalo': 0.9118282986366658,
 'Salex': 0.9374180131412205,
 'Ser': 0.9762760896075,
 'T6C': 0.9405611264348838,
 'Woodyi': 0.9422289557910769}

### Experiment 1: Glucose
Define a medium where glucose is the only carbon source.

In [5]:
# Define a larger "minimal" media
gluc_medium = gem2cue.utils.Media({
 'EX_aso3_e': 1000.0,
 'EX_aso4_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 1000.0, # Glucose
 'EX_glc__aD_e': 1000.0, # Alpha glucose
 'EX_h2_e': 1000.0,
 'EX_h2o2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h2s_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_no2_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_no_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_ppi_e': 1000.0, # Diphosphate
 'EX_sel_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_so3_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tsul_e': 1000.0,
 'EX_zn2_e': 1000.0})

In [6]:
# Make an experiment for each Strain
glucose_experiments = [gem2cue.utils.Experiment(strain, media=gluc_medium) for strain in strain_list]

In [10]:
glucose_experiments[0].strain.model.medium

{'EX_aso3_e': 1000.0,
 'EX_aso4_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 1000.0,
 'EX_h2_e': 1000.0,
 'EX_h2o2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h2s_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_no2_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_no_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so3_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tsul_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [11]:
glucose_experiments[0].select_media()

In [12]:
glucose_experiments[0].strain.model.medium

{'EX_aso3_e': 1000.0,
 'EX_aso4_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 1000.0,
 'EX_h2_e': 1000.0,
 'EX_h2o2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h2s_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_no2_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_no_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so3_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tsul_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [13]:
# Calculate CUE for all of the Experiments
for exp in glucose_experiments:
    exp.CUE(ex_nomenclature = {'C_e'})

/home/helen/Documents/PhD/Segre Lab/GEM2CUE/gem2cue/utils.py:109: UserWarning: There is already a cue value saved to this experiment, running will overwrite those results
  warnings.warn('There is already a cue value saved to this experiment, running will overwrite those results')
/home/helen/Documents/PhD/Segre Lab/GEM2CUE/gem2cue/utils.py:109: UserWarning: There is already a cue value saved to this experiment, running will overwrite those results
  warnings.warn('There is already a cue value saved to this experiment, running will overwrite those results')
/home/helen/Documents/PhD/Segre Lab/GEM2CUE/gem2cue/utils.py:109: UserWarning: There is already a cue value saved to this experiment, running will overwrite those results
  warnings.warn('There is already a cue value saved to this experiment, running will overwrite those results')
/home/helen/Documents/PhD/Segre Lab/GEM2CUE/gem2cue/utils.py:109: UserWarning: There is already a cue value saved to this experiment, running will overwri

In [14]:
glucose_experiments[0].strain.model.medium

{'EX_aso3_e': 1000.0,
 'EX_aso4_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 1000.0,
 'EX_h2_e': 1000.0,
 'EX_h2o2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h2s_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_no2_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_no_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so3_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tsul_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [15]:
[exp.cue for exp in glucose_experiments]

[0.6865607774050599,
 0.7017981841156299,
 0.6878920803972508,
 None,
 None,
 0.8353794723276078,
 0.7499679334312341,
 None,
 0.8470794854270659,
 0.841940199251723,
 0.7492616902585909,
 0.7778933161683088,
 None,
 0.7443779588693178,
 0.5617618691024199,
 0.8333333333333334,
 0.6879654847916286,
 0.8333333333333334,
 0.6904566834618701,
 0.8333333333333334]

### Experiment 2: Acetate

In [ ]:
# Copy the medium from one of the models
ace_media = strain_list[0].model.medium
# It already has acetate, just need to remove glucose
ace_media['EX_glc__D_e'] = 0

In [ ]:
# Make an experiment for each Strain
ace_experiments = [gem2cue.utils.Experiment(strain, ace_media) for strain in strain_list]

In [ ]:
# Calculate CUE for all of the Experiments
for exp in ace_experiments:
    exp.CUE(ex_nomenclature = {'C_e'})

In [ ]:
[exp.cue for exp in ace_experiments]

## Visualization